In [ ]:
pip install hazm

In [ ]:
pip install urlextract

In [ ]:
pip install kafka-python

In [1]:
from kafka import KafkaConsumer
from json import loads
from time import sleep
from json import dumps
from kafka import KafkaProducer
import requests
import time
import re
import json

from urlextract import URLExtract
import json
import pandas as pd
from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer

tagger = POSTagger(model='/mnt/d/BigData/resources/postagger.model')


In [2]:
special_keywords = ['بورس','اقتصاد','تحریم','دولت','حسن روحانی','انتخابات','دلار','طلا','کرونا','تورم','دانشگاه','کووید۱۹','کوید۱۹','کویید۱۹','کویید ۱۹','کوید ۱۹']
special_keywords = set(special_keywords)
stop_word_set = set(stopwords_list())
def extract_verbs(tokenized_sentence):
    verb_list = set()
    for word,tag in tokenized_sentence:
        if tag=='V':
            verb_list.add(word)
    return list(verb_list)

def keyWordExtraction(doc):
    text=doc['content']
    normalizer = Normalizer()
    vectorizer = TfidfVectorizer(lowercase=False, preprocessor=normalizer.normalize, tokenizer=word_tokenize,
                                 stop_words=stopwords_list())
    tokenized_sentence = tagger.tag(word_tokenize(text.replace('#','')))
    verbs = extract_verbs(tokenized_sentence)
    for v in verbs: # Remove Verbs From Text
        text = text.replace(v,'')
    vectors = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names()
    denselist = vectors.todense().tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    
    sp_set = set()
    for cl in df.columns:
        if cl.replace('#','') in special_keywords:
            df[cl]=0
            sp_set.add(cl.replace('#',''))
    df.sort_values(by=0, axis=1, ascending=False, inplace=True, na_position='last')
    max = df.values.size
    min = int(max * 0.70)
    keywords = list(df.keys()[min:max].values)
    keywords = [x.replace('#','') for x in keywords]
    keywords = [x for x in keywords if len(x) >= 2 and x not in stop_word_set]
    doc['keywords']= list(set(keywords+ list(sp_set) ))
    return doc


In [3]:
consumer = KafkaConsumer(
    'PreProcess',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [4]:
extractor = URLExtract()
def preprocess(tweet):
    tweet['hashtags'] = re.findall(r"#(\w+)",tweet['content'])
    tweet['links']= extractor.find_urls(tweet['content'])
    tweet = keyWordExtraction(tweet)
    return tweet

In [5]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],key_serializer=str.encode,
                         value_serializer=lambda x:dumps(x).encode('utf-8'))

In [6]:
cnt=0
ext_counter=0

In [8]:
for message in consumer:
    cnt+=1
#     if cnt < 100:
#         continue
    if cnt%50 ==0:
        print(cnt)
    message = message.value
    if 'content' not in message or len(message['content'])< 10:
        continue
#     print('t2')
    try:
        tweet = preprocess(message)
    #     print('t3')
        producer.send('persistence', key=tweet['id'],value=tweet)
        producer.send('statics', key=tweet['id'],value=tweet)
    except:
        ext_counter+=1
        if ext_counter % 50 ==0:
            print("An exception occurred: "+str(ext_counter)) 


3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750



KeyboardInterrupt

